# Electric Network Reconstruction System ENRS
## TFM 2025  
### Autor : Andres Felipe Vargas Nuñez 
![](../Static_files/logo_ucm_ntic.png )

##### Este archivo genera escenarios de consumo eléctricos en redes bajo diferentes configuraciones de redes eléctricas de tipo radial.

In [4]:
import os 
import uuid
import random
from datetime import datetime

import numpy as np
import pandas as pd
import pandapower as pp
import matplotlib.pyplot as plt
import pandapower.networks as nw

import pandapower.plotting as plt
import pandapower.timeseries as ts
from pandapower.control import ConstControl
from pandapower.timeseries.data_sources.frame_data import DFData

In [1]:
def create_custom_network(num_buses, num_loads, connection_type="tree"):
    # Crear una red vacía
    net = pp.create_empty_network()
    buses = [pp.create_bus(net, vn_kv=110, name=f"Bus {i+1}") for i in range(num_buses)]

    connected_buses = [buses[0]]
    std_types = pp.available_std_types(net, element='line')
    if connection_type == "tree":
        # for i in range(1, num_buses):
        #     parent_bus = random.choice(buses[:i])
        #     configuracion_aleatoria = random.choice(list(std_types.index))
        #     pp.create_line(net,name=f"line {i+1}" ,from_bus=parent_bus, to_bus=buses[i], length_km=(random.uniform(0.2, 10.0)), std_type=configuracion_aleatoria)
        # Crear conexiones de líneas de forma aleatoria para formar una red radial
        for _ in range(1, num_buses):
            available_buses = [bus for bus in buses if bus not in connected_buses]
            new_bus = random.choice(available_buses)
            parent_bus = random.choice(connected_buses)
            configuracion_aleatoria = random.choice(list(std_types.index))
            pp.create_line(net, name=f"line {len(net.line) + 1}", from_bus=parent_bus, to_bus=new_bus, length_km=random.uniform(0.2, 10.0), std_type=configuracion_aleatoria)
            connected_buses.append(new_bus)

    elif connection_type == "mesh":
        for i in range(1, num_buses):
            for j in range(i):
                if random.random() < 0.5:  
                    configuracion_aleatoria = random.choice(list(std_types.index))
                    pp.create_line(net, from_bus=buses[i], to_bus=buses[j], length_km=(random.uniform(0.2, 10.0)), std_type=configuracion_aleatoria)
                    
    elif connection_type == "ring":
        for i in range(num_buses):
            configuracion_aleatoria = random.choice(list(std_types.index))
            pp.create_line(net, from_bus=buses[i], to_bus=buses[(i+1) % num_buses], length_km=(random.uniform(0.2, 10.0)), std_type=configuracion_aleatoria)

            if random.random() < 0.3:  
                distant_bus = random.choice(buses)
                if distant_bus != buses[i] and distant_bus != buses[(i+1) % num_buses]:
                    pp.create_line(net, from_bus=buses[i], to_bus=distant_bus, length_km=(random.uniform(0.2, 10.0)), std_type=configuracion_aleatoria)

    else:
        raise ValueError("Tipo de conexión no válido. Usa 'tree' o 'ring'.")


    nodos_finales = [bus for bus in buses if len(net.line[(net.line.from_bus == bus) | (net.line.to_bus == bus)]) == 1]
    print(nodos_finales)
    num_loads = len(nodos_finales)  # Puedes ajustar esto según tus necesidades
    loads = [
        pp.create_load(net, bus=bus,
                    p_mw=random.uniform(1.0, 5.0), 
                    q_mvar=random.uniform(0.5, 2.5), 
                    name=f"Load {i+1}") 
        for i, bus in enumerate(nodos_finales)
    ]

    slack_bus = 0
    pp.create_gen(net, bus=slack_bus, p_mw=random.uniform(5.0, 10.0), vm_pu=1.02, slack=True, name="Slack Gen")
    pp.create_ext_grid(net, bus=slack_bus, vm_pu=1.02)

    return net

def create_network(net_type=0):
    if net_type == 0:
        net =nw.simple_four_bus_system()
    elif net_type == 1:   
        net =nw.case24_ieee_rts()
    elif net_type == 2:
        net = nw.case30()
    else:
        net = nw.case14()
    return net

def calculate_resistance(distance_km):
    resistivity = 0.017 
    area = 1.0  
    resistance = (resistivity / area) * distance_km
    return resistance

def configure_output_writer(net, output_path):
    return ts.OutputWriter(net, output_path=output_path, output_file_type=".xlsx")

# def create_profile_data(time_steps, net):
#     profile_data = np.random.rand(time_steps, len(net.load))
#     return ts.DFData(pd.DataFrame(profile_data))

def create_profile_data(time_steps, net, pattern='daily'):
    # Crear perfiles base para diferentes tipos de cargas
    if pattern == 'daily':
        base_profile = np.sin(np.linspace(0, 2 * np.pi, time_steps)) + 1  # Variación diaria
    elif pattern == 'weekly':
        base_profile = np.sin(np.linspace(0, 4 * np.pi, time_steps)) + 1  # Variación semanal
    elif pattern == 'random_walk':
        base_profile = np.cumsum(np.random.randn(time_steps))  # Random walk
    else:
        base_profile = np.ones(time_steps)  # Perfil constante

    # Crear perfiles de carga para cada carga en la red
    profile_data = np.zeros((time_steps, len(net.load)))
    for i in range(len(net.load)):
        noise = np.random.normal(0, 0.1, time_steps)  # Agregar algo de ruido
        profile_data[:, i] = base_profile + noise

    # Normalizar perfiles para obtener valores más realistas
    profile_data = np.clip(profile_data, 0, None)  # Clipping to avoid negative values
    profile_data /= profile_data.max(axis=0)  # Normalize each load profile
    profile_data *= np.random.uniform(1.0, 5.0, (1, len(net.load)))  # Scale profiles

    return ts.DFData(pd.DataFrame(profile_data))


def configure_constant_load_control(net, ds):
    return ConstControl(net, element='load', element_index=net.load.index, variable='scaling', data_source=ds, profile_name=net.load.index)

def log_variables(ow):
    ow.log_variable('res_bus', 'vm_pu')
    ow.log_variable('res_bus', 'va_degree')
    ow.log_variable('res_bus', 'p_mw')
    ow.log_variable('res_bus', 'q_mvar')

def run_simulation(net, time_steps) -> bool:
    try:
        ts.run_timeseries(net, time_steps)
        return True
    except Exception as e:
        print(f"Error durante la ejecución de series temporales: {e}")
        return False

def load_results(output_path):
    res_bus = pd.read_excel(f'{output_path}/res_bus/vm_pu.xlsx', header=None)
    res_load_p_mw = pd.read_excel(f'{output_path}/res_load/p_mw.xlsx', header=None)
    res_load_q_mvar = pd.read_excel(f'{output_path}/res_load/q_mvar.xlsx', header=None)
    res_line = pd.read_excel(f'{output_path}/res_line/loading_percent.xlsx')
    lines_config = pd.read_csv(f'{output_path}/line_configuration.csv', header=0)
    loads_config = pd.read_csv(f'{output_path}/loads_configuration.csv', header=0)
    return res_bus, res_load_p_mw, res_load_q_mvar, res_line , lines_config, loads_config

def format_load_data(res_load, load_bus_ids, value_name):
    res_load.columns = ['time_step'] + [f'{bus}' for bus in load_bus_ids]
    res_load = res_load[~res_load['time_step'].isna()]
    res_load = res_load.reset_index().melt(id_vars=['index'], var_name='node_id', value_name=value_name)
    res_load = res_load.query("node_id != 'time_step'")
    return res_load

def format_bus_data(res_bus, max_range):
    res_bus.columns = ['time_step', 'line_id'] + [f'{bus}' for bus in range(1, max_range)]
    res_bus = res_bus[~res_bus['time_step'].isna()]
    res_bus = res_bus.reset_index().melt(id_vars=['index'], var_name='node_id', value_name='vm_pu')
    res_bus = res_bus.query("node_id != 'time_step' & node_id != 'line_id'")
    return res_bus

def merge_data(res_bus, res_load_p_mw, res_load_q_mvar):
    res_load_p_mw.set_index(['index', 'node_id'], inplace=True)
    res_bus.set_index(['index', 'node_id'], inplace=True)
    res_load_q_mvar.set_index(['index', 'node_id'], inplace=True)
    df_combined = res_bus.join(res_load_p_mw, on=['index', 'node_id'] ,lsuffix='__bus', rsuffix='_loadp') 
    df_combined = df_combined.join(res_load_q_mvar, on=['index', 'node_id'], how="left",rsuffix='_loadq')
    return df_combined

def create_incidence_matrix(net):
    incidence_matrix = pp.topology.create_nxgraph(net)
    return pd.DataFrame(nw.to_pandas_adjacency(incidence_matrix, nodelist=net.bus.index, weight='weight'))

def create_line_data(net):
    line_data = []
    for line in net.line.index:
        from_bus = net.line.from_bus.at[line]
        to_bus = net.line.to_bus.at[line]
        length_km = net.line.length_km.at[line]
        r_ohm_per_km = net.line.r_ohm_per_km.at[line]
        line_data.append({'line_id': line, 'from_bus': from_bus, 'to_bus': to_bus, 'length_km': length_km, 'r_ohm_per_km': r_ohm_per_km})
    return pd.DataFrame(line_data)

def print_inciden_matrix(df):
    df = df.applymap(lambda x: 0 if x == '0' or x == 0 else 1)

def incidence_matrix(line_df):
    from_buses = line_df['from_bus']
    to_buses = line_df['to_bus']
    length_kms = line_df['length_km']
    r_ohm_per_kms = line_df['r_ohm_per_km']
    num_buses = max(from_buses.max(), to_buses.max()) + 1
    num_lines = len(line_df)
    incidence_matrix = np.zeros((num_buses, num_lines), dtype=tuple)
    # (length_km,r_ohm_per_km
    for i, (from_bus, to_bus,length_km,r_ohm_per_km) in enumerate(zip(from_buses, to_buses,length_kms,r_ohm_per_kms)):
        incidence_matrix[from_bus, i] = 1
        incidence_matrix[to_bus, i] = 1

    df = pd.DataFrame(incidence_matrix, columns=[i for i in range(num_lines)], index=[j for j in range(num_buses)])

    return df

def plot_simple_df_net(df, is_print_lines=False):
    df = df.transpose()
    net = pp.create_empty_network()
    buses = [pp.create_bus(net, vn_kv=110, name=f"Bus {bus}") for bus in range(len(df.columns))]
    number_line = 0

    for i, row in df.iterrows():
        from_bus = None
        to_bus = None
        for j, value in row.items():
            # print(f" {type(value)}   {value} " )
            value = eval(value) if isinstance(value,str)else value
            if value != 0 and  from_bus is None:
                from_bus = int(j)
            elif value != 0 and  to_bus is None:
                to_bus = int(j)
                if (from_bus != to_bus ):
                    if isinstance(value,str):
                        length_km = eval(value)[0]
                    else:
                        length_km = 10.0
                    pp.create_line(net,name=f"number_line{number_line}" ,from_bus=from_bus, to_bus=to_bus, length_km=length_km, std_type="NAYY 4x50 SE")
                    number_line +=1

    slack_bus = 0 
    pp.create_gen(net, bus=slack_bus, p_mw=random.uniform(5.0, 10.0), vm_pu=1.02, slack=True, name="Slack Gen")
    other_gen_buses = random.sample([b for b in buses if b != slack_bus], k=max(1,  len(df.index) // 4))
    for bus in other_gen_buses:
        pp.create_gen(net, bus=bus, p_mw=random.uniform(3.0, 6.0), vm_pu=1.02, name=f"Gen {bus}")

    pp.create_ext_grid(net, bus=slack_bus, vm_pu=1.02)
    plt.simple_plot(net)
    plt.show()
    if is_print_lines:
        print(net.line)

In [3]:
def generate_scenario_data(base_output_path:str ,nodes:int,loads:int,num_scenarios:int=10, time_steps:int=10):
    net = create_network()
    net_name = f"net_generate_n_{nodes}"

    if not os.path.exists(base_output_path):
        os.makedirs(base_output_path)
       
    for scenario in range(num_scenarios):
        output_path = f"{base_output_path}/case_{net_name}_scenario_{scenario}"
        if not os.path.exists(output_path):
            os.makedirs(output_path)
        
        available_buses = list(net.bus.index)
        available_buses.remove(0)
        if scenario>=0:
            num_buses = nodes 
            num_loads = loads  # Número de cargas
            connection_type = "tree"  # O "ring" para una red en anillo
            net = create_custom_network(num_buses, num_loads, connection_type)
            
        ow = configure_output_writer(net, output_path)
        ds = create_profile_data(time_steps, net)
        configure_constant_load_control(net, ds)
        log_variables(ow)
        run_simulation(net, time_steps)
        net.line.to_csv(f"{output_path}/line_configuration.csv", index=False )
        net.load.to_csv(f"{output_path}/loads_configuration.csv", index=False )
        


In [5]:
max_size_nodes = 24
nodes = 12
lines = nodes-1
loads = 9
num_intervals = 48
num_scenarios = 2
num_networks = 1
time_steps_simulation = num_intervals * 48
base_output_path = f"../Data-3-test"
base_output_net_data_path=f"{base_output_path}/net_data/AV1-test"
# uniq_number_nodes = random.sample(range(12, 25), num_networks)  # 13 números únicos en el rango 12-24
# print(uniq_number_nodes)
for i in range(20,23):
    generate_scenario_data(base_output_net_data_path,i,loads ,num_scenarios, time_steps_simulation )


[1, 2, 7, 8, 11, 14, 15, 19]


No time steps to calculate are specified. I'll check the datasource of the first controller for avaiable time steps
c:\Users\afgvn\Documents\backup\UCM-2024\TFM\env\Lib\site-packages\pandapower\timeseries\output_writer.py:177: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'range(0, 2304)' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  self.output["Parameters"].loc[:, "time_step"] = self.time_steps
c:\Users\afgvn\Documents\backup\UCM-2024\TFM\env\Lib\site-packages\pandapower\control\run_control.py:50: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  level = controller.level.fillna(0).apply(asarray).values
100%|██████████| 2304/2304 [00:19<00:00, 118.74it/s]


[1, 5, 6, 10, 11, 12, 14, 17, 18]


No time steps to calculate are specified. I'll check the datasource of the first controller for avaiable time steps
c:\Users\afgvn\Documents\backup\UCM-2024\TFM\env\Lib\site-packages\pandapower\timeseries\output_writer.py:177: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'range(0, 2304)' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  self.output["Parameters"].loc[:, "time_step"] = self.time_steps
c:\Users\afgvn\Documents\backup\UCM-2024\TFM\env\Lib\site-packages\pandapower\control\run_control.py:50: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  level = controller.level.fillna(0).apply(asarray).values
100%|██████████| 2304/2304 [00:13<00:00, 164.88it/s]


[1, 2, 5, 8, 9, 10, 11, 12, 15, 18, 20]


...

No time steps to calculate are specified. I'll check the datasource of the first controller for avaiable time steps
c:\Users\afgvn\Documents\backup\UCM-2024\TFM\env\Lib\site-packages\pandapower\timeseries\output_writer.py:177: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'range(0, 2304)' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  self.output["Parameters"].loc[:, "time_step"] = self.time_steps
c:\Users\afgvn\Documents\backup\UCM-2024\TFM\env\Lib\site-packages\pandapower\control\run_control.py:50: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  level = controller.level.fillna(0).apply(asarray).values
100%|██████████| 2304/2304 [00:12<00:00, 189.08it/s]


[2, 3, 4, 6, 8, 9, 10, 12, 13, 19, 20, 21]


No time steps to calculate are specified. I'll check the datasource of the first controller for avaiable time steps
c:\Users\afgvn\Documents\backup\UCM-2024\TFM\env\Lib\site-packages\pandapower\timeseries\output_writer.py:177: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'range(0, 2304)' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  self.output["Parameters"].loc[:, "time_step"] = self.time_steps
c:\Users\afgvn\Documents\backup\UCM-2024\TFM\env\Lib\site-packages\pandapower\control\run_control.py:50: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  level = controller.level.fillna(0).apply(asarray).values
100%|██████████| 2304/2304 [00:12<00:00, 188.66it/s]


[2, 5, 6, 10, 11, 12, 13, 15, 16, 17, 21]


No time steps to calculate are specified. I'll check the datasource of the first controller for avaiable time steps
c:\Users\afgvn\Documents\backup\UCM-2024\TFM\env\Lib\site-packages\pandapower\timeseries\output_writer.py:177: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'range(0, 2304)' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  self.output["Parameters"].loc[:, "time_step"] = self.time_steps
c:\Users\afgvn\Documents\backup\UCM-2024\TFM\env\Lib\site-packages\pandapower\control\run_control.py:50: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  level = controller.level.fillna(0).apply(asarray).values
100%|██████████| 2304/2304 [00:12<00:00, 187.37it/s]
